In [40]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import spacy
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from collections import Counter

In [41]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

# Dataframe work

In [42]:
df_full = pd.read_csv("/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/sentiment_analysis_all.csv")

In [148]:
df_full['Party'] = df_full['Party'].replace('Bündnis 90/Die Grünen', 'Die Grünen')

## Heatmap

heatmap of top 5 speakers

In [47]:
sentence_count = df_full.groupby('Name').size().sort_values(ascending=False)

# Get the top 5 speakers
top_speakers = sentence_count.head(5).index

# Filter the original dataframe for only the top 5 speakers
# and select only 'Name' and 'Text_Spoken' columns
example_df = df_full[df_full['Name'].isin(top_speakers)][['Name', 'Text_Spoken']]

# Display the resulting dataframe
example_df

,Name,Text_Spoken
0,Wolfgang Kubicki,Der Antrag ist damit abgelehnt.
1,Wolfgang Kubicki,Endgültiges Ergebnis Abgegebene Stimmen: 633;d...
2,Wolfgang Kubicki,Sybille Benning Dr André Berghegger Melanie Be...
3,Wolfgang Kubicki,Dr Reinhard Brandl Sebastian Brehm Heike Brehm...
4,Wolfgang Kubicki,Fischer (Karlsruhe Land).
...,...,...
4530563,Petra Pau,"Frau Kollegin, achten Sie bitte auf die Zeit."
4530566,Petra Pau,"Sie haben alle Chancen, das nachher zum nächst..."
4530567,Petra Pau,Sie müssen jetzt zum Punkt kommen.
4530573,Petra Pau,Das Wort hat die Kollegin Kerstin Tack für die...


In [48]:
# Encode the sentences to get the embeddings
example_df['Embeddings'] = example_df['Text_Spoken'].apply(lambda x: model.encode(x))

# Group by speaker and average their sentence embeddings
speaker_embeddings = example_df.groupby('Speaker')['Embeddings'].apply(lambda x: x.mean())

# Calculate the cosine similarity between each speaker's average embedding
similarity_matrix = cosine_similarity(list(speaker_embeddings))

# Convert the similarity matrix into a dataframe
similarity_df = pd.DataFrame(similarity_matrix, index=speaker_embeddings.index, columns=speaker_embeddings.index)

# Visualize the similarity matrix
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_df, annot=True)
plt.title('Speaker Similarity Heatmap')
plt.savefig('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Präsentation/Graphs/heatmap.png')
plt.show();

KeyboardInterrupt: 

# Spacy model

In [49]:
nlp = spacy.load('de_core_news_lg')

# Let us add the Party

In [50]:
df1 = df_full.copy()

In [ ]:
# Chunky chunk chunk
# Group by speaker and concatenate all text spoken by the same speaker
grouped_df = df1.groupby('Name')['Text_Spoken'].apply(' '.join).reset_index()

# Add the 'Party' column to the grouped_df
grouped_df = grouped_df.merge(df1[['Name', 'Party']].drop_duplicates(), on='Name')

# Function to process text in chunks
def process_text_in_chunks(text, chunk_size=1000000):
    # Process the text in chunks of the given size
    doc = nlp(text[:chunk_size])
    processed_text = ' '.join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha])
    for i in range(chunk_size, len(text), chunk_size):
        doc = nlp(text[i:i+chunk_size])
        processed_text += ' ' + ' '.join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha])
    return processed_text

# Apply preprocessing to the grouped DataFrame
grouped_df['Processed_Text'] = grouped_df['Text_Spoken'].apply(lambda text: process_text_in_chunks(text))

# Vectorize the text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_df['Processed_Text'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert cosine similarity matrix to DataFrame
cosine_sim_df_text = pd.DataFrame(cosine_sim, index=grouped_df['Name'], columns=grouped_df['Name'])

# Display the similarity matrix
cosine_sim_df_text

In [ ]:
# Save the cosine similarity matrix to a CSV file
cosine_sim_df_text.to_csv('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/cosine_similarity_matrix.csv')

In [ ]:
party_colors_map = {
    'SPD': 'red',
    'PDS': '#801818',
    'CDU': 'white',
    'FDP': 'yellow',
    'Die Grünen': 'green',
    'CSU': 'white',
    'AfD': '#0489DB',
    'Die Linke': 'pink',
    'fraktionslos': 'white',
    'LKR': '#FE8100',
    'SSW': '#00277E'
}

In [ ]:
# Initialize the graph
G = nx.Graph()

# Add nodes and associate each node with a party
for index, row in grouped_df.iterrows():
    G.add_node(row['Name'], party=row['Party'])

# Get a list of colors for each node
node_colors = party_colors_map

# Define a threshold for similarity to draw an edge
threshold = 0.6  # You may need to adjust this value

# Add edges to the graph
for i, speaker1 in enumerate(cosine_sim_df_text.index):
    for j, speaker2 in enumerate(cosine_sim_df_text.columns):
        if i < j:  # This ensures that each pair is only considered once
            similarity = cosine_sim_df_text.iloc[i, j]
            if similarity > threshold:
                G.add_edge(speaker1, speaker2, weight=similarity)

# Draw the graph
plt.figure(figsize=(20, 10), facecolor='#282828')
pos = nx.kamada_kawai_layout(G)
#pos = nx.spring_layout(G, k=0.1)  # k is the optimal distance between nodes, may need to be adjusted

# Create a subgraph of G that only includes the largest connected component
# If you want to include smaller connected components as well, you can adjust this part
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc)

# Draw only the nodes and edges from the largest connected component
nx.draw_networkx_nodes(subgraph, pos, node_color=[party_colors_map[subgraph.nodes[speaker]['party']] for speaker in subgraph.nodes], alpha=0.7, node_size=590)  # Adjust node_size as needed
nx.draw_networkx_edges(subgraph, pos, edge_color='gray', alpha=0.5, width=0.5)


# Create a legend for the color map
party_labels = list(party_colors_map.keys())
patches = [plt.Line2D([0], [0], marker='o', color='w', label=party,
                      markersize=10, markerfacecolor=color) for party, color in party_colors_map.items()]
plt.legend(handles=patches, title="Political Parties", loc='upper left', bbox_to_anchor=(1, 1), fontsize='large')

plt.title('Network Graph of Speaker Similarity',  fontsize=20, loc='center', color='white')
plt.axis('off')
plt.savefig('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Präsentation/Graphs/speaker_similarity_graph_1.png', dpi=300, bbox_inches='tight', facecolor='#282828')
plt.show();

# Networkgraph Based on Sentiment

In [ ]:
# Group by 'Name' and calculate the mean of each sentiment
grouped_sentiment_df = df1.groupby('Name')[['positive', 'neutral', 'negative']].mean().reset_index()

# Add the 'Party' column to the grouped_df before creating the sentiment matrix
grouped_sentiment_df = grouped_sentiment_df.merge(df1[['Name', 'Party']].drop_duplicates(), on='Name')

# Create the sentiment matrix from the grouped DataFrame
sentiment_matrix = grouped_sentiment_df[['positive', 'neutral', 'negative']].values

# Calculate cosine similarity
cosine_sim_sentiment = cosine_similarity(sentiment_matrix, sentiment_matrix)

# Convert cosine similarity matrix to DataFrame
cosine_sim_df_sentiment = pd.DataFrame(cosine_sim_sentiment, 
                                       index=grouped_sentiment_df['Name'], 
                                       columns=grouped_sentiment_df['Name'])

# Display the similarity matrix
cosine_sim_df_sentiment

In [ ]:
# Save the cosine similarity matrix to a CSV file
cosine_sim_df_sentiment.to_csv('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/cosine_sim_df_sentiment.csv')

In [ ]:
# Initialize the graph
G = nx.Graph()

# Add nodes and associate each node with a party
for index, row in grouped_sentiment_df.iterrows():
    G.add_node(row['Name'], party=row['Party'])

# Get a list of colors for each node
node_colors = party_colors_map

# Define a threshold for similarity to draw an edge
threshold = 0.999  # You may need to adjust this value

# Add edges to the graph
for i, speaker1 in enumerate(cosine_sim_df_sentiment.index):
    for j, speaker2 in enumerate(cosine_sim_df_sentiment.columns):
        if i < j:  # This ensures that each pair is only considered once
            similarity = cosine_sim_df_sentiment.iloc[i, j]
            if similarity > threshold:
                G.add_edge(speaker1, speaker2, weight=similarity)

# Draw the graph
plt.figure(figsize=(20, 10), facecolor='#282828')
pos = nx.kamada_kawai_layout(G)
#pos = nx.spring_layout(G, k=0.1)  # k is the optimal distance between nodes, may need to be adjusted

# Create a subgraph of G that only includes the largest connected component
# If you want to include smaller connected components as well, you can adjust this part
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc)

# Draw only the nodes and edges from the largest connected component
nx.draw_networkx_nodes(subgraph, pos, node_color=[party_colors_map[subgraph.nodes[speaker]['party']] for speaker in subgraph.nodes], alpha=0.7, node_size=100)  # Adjust node_size as needed
nx.draw_networkx_edges(subgraph, pos, edge_color='gray', alpha=0.5, width=0.5)


# Create a legend for the color map
party_labels = list(party_colors_map.keys())
patches = [plt.Line2D([0], [0], marker='o', color='w', label=party,
                      markersize=10, markerfacecolor=color) for party, color in party_colors_map.items()]
plt.legend(handles=patches, title="Political Parties", loc='upper left', bbox_to_anchor=(1, 1), fontsize='large')

plt.title('Network Graph of Speaker Similarity',  fontsize=20, loc='center', color='white')
plt.axis('off')
plt.savefig('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Präsentation/Graphs/4.png', dpi=300, bbox_inches='tight', facecolor='#282828')
plt.show();